# Image Classification Transfer learning with YOLOv11
![yolo](https://cdn.prod.website-files.com/680a070c3b99253410dd3df5/680a070c3b99253410dd4791_67ed5670d7ecbda0527fe8b3_66f680814693dd5c3b60dfcb_YOLO11_Thumbnail.png)

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 781.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
import torchvision.transforms as T

from ultralytics import YOLO
from ultralytics.data.dataset import ClassificationDataset
from ultralytics.models.yolo.classify import (
    ClassificationTrainer,
    ClassificationValidator,
)  # Import ClassificationValidator


class CustomizedDataset(ClassificationDataset):
    """A customized dataset class for image classification with enhanced data augmentation transforms."""

    def __init__(self, root: str, args, augment: bool = False, prefix: str = ""):
        """Initialize a customized classification dataset with enhanced data augmentation transforms."""
        super().__init__(root, args, augment, prefix)
        train_transforms = T.Compose(
            [
                T.Resize((args.imgsz, args.imgsz)),
                T.RandomHorizontalFlip(p=args.fliplr),
                T.RandomVerticalFlip(p=args.flipud),
                T.RandAugment(interpolation=T.InterpolationMode.BILINEAR),
                T.ColorJitter(
                    brightness=args.hsv_v,
                    contrast=args.hsv_v,
                    saturation=args.hsv_s,
                    hue=args.hsv_h,
                ),
                T.ToTensor(),
                T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
                T.RandomErasing(p=args.erasing, inplace=True),
            ]
        )
        val_transforms = T.Compose(
            [
                T.Resize((args.imgsz, args.imgsz)),
                T.ToTensor(),
                T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
            ]
        )
        self.torch_transforms = train_transforms if augment else val_transforms


class CustomizedTrainer(ClassificationTrainer):
    """A customized trainer class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train", batch=None):
        """Build a customized dataset for classification training and the validation during training."""
        return CustomizedDataset(root=img_path, args=self.args, augment=mode == "train", prefix=mode)


class CustomizedValidator(ClassificationValidator):  # Now ClassificationValidator is defined
    """A customized validator class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train"):
        """Build a customized dataset for classification standalone validation."""
        return CustomizedDataset(root=img_path, args=self.args, augment=mode == "train", prefix=self.args.split)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Download Dataset

In [ ]:
import gdown

# Google Drive file ID
file_id = "1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl"
# Download destination filename
output = "myfile.zip"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl
From (redirected): https://drive.google.com/uc?id=1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl&confirm=t&uuid=15418a89-d8f1-4650-9d52-358f9e5dc362
To: /content/myfile.zip
100%|██████████| 63.9M/63.9M [00:00<00:00, 170MB/s]


'myfile.zip'

In [ ]:
import zipfile

with zipfile.ZipFile("myfile.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

### Custom Training or Transfer learning

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt")
#model.train(data="imagenet1000", trainer=CustomizedTrainer, epochs=10, imgsz=224, batch=64)
model.train(data="/content/dataset/train", trainer=CustomizedTrainer, epochs=10, imgsz=224, batch=64)

100%|██████████| 5.52M/5.52M [00:00<00:00, 116MB/s]


Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/train, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

100%|██████████| 5.35M/5.35M [00:00<00:00, 102MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1430.6±443.0 MB/s, size: 40.5 KB)


train: Scanning /content/dataset/train_split/train... 1019 images, 0 corrupt: 100%|██████████| 1019/1019 [00:00<00:00, 5720.12it/s]

...

                   all      0.969          1



10 epochs completed in 0.035 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.2MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /content/dataset/train/train
Found 1275 images in subdirectories. Attempting to split...
Splitting /content/dataset/train (2 classes, 1275 images) into 80% train, 20% val...
Split complete in /content/dataset/train_split ✅
train: /content/dataset/train_split/train... found 1214 images in 2 classes ✅ 
val: /content/dataset/train_split/val... found 451 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  9.77it/s]


                   all      0.969          1
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f099b9c5b50>
curves: []
curves_results: []
fitness: 0.984375
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.96875, 'metrics/accuracy_top5': 1.0, 'fitness': 0.984375}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.07936589843771102, 'inference': 0.20452327734377107, 'loss': 8.314843746859424e-05, 'postprocess': 0.00013074609350738342}
task: 'classify'
top1: 0.96875
top5: 1.0

In [ ]:
# CLI Start training from a pretrained *.pt model
#!yolo classify train data=imagenet10 model=yolo11n-cls.pt epochs=5 imgsz=224

In [ ]:
metrics = model.val(data="/content/dataset/valid", validator=CustomizedValidator, imgsz=224, batch=64)

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /content/dataset/valid/train
Found 364 images in subdirectories. Attempting to split...
Splitting /content/dataset/valid (2 classes, 364 images) into 80% train, 20% val...
Split complete in /content/dataset/valid_split ✅
train: /content/dataset/valid_split/train... found 290 images in 2 classes ✅ 
val: /content/dataset/valid_split/val... found 74 images in 2 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 720.4±250.4 MB/s, size: 25.3 KB)


val: Scanning /content/dataset/valid_split/val... 74 images, 0 corrupt: 100%|██████████| 74/74 [00:00<00:00, 5338.65it/s]

val: New cache created: /content/dataset/valid_split/val.cache



               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


                   all      0.905          1
Speed: 0.6ms preprocess, 4.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [ ]:
metrics.top1  # top1 accuracy


0.9054054021835327

In [ ]:
metrics.top5  # top5 accuracy

1.0

In [ ]:
model = YOLO("/content/runs/classify/train/weights/best.pt")  # load a custom model

# Predict with the model
results = model("/content/dataset/test/daisy/14333681205_a07c9f1752_m_jpg.rf.6ff96d9fe33f0bd19a18425f32d470b1.jpg")


image 1/1 /content/dataset/test/daisy/14333681205_a07c9f1752_m_jpg.rf.6ff96d9fe33f0bd19a18425f32d470b1.jpg: 224x224 daisy 1.00, dandelion 0.00, 6.5ms
Speed: 4.5ms preprocess, 6.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


In [ ]:
# Predict with the model
results = model("/content/dataset/test/dandelion/16159487_3a6615a565_n_jpg.rf.6d473a1fe680a3e930f3ff28464c46a9.jpg")


image 1/1 /content/dataset/test/dandelion/16159487_3a6615a565_n_jpg.rf.6d473a1fe680a3e930f3ff28464c46a9.jpg: 224x224 dandelion 1.00, daisy 0.00, 6.2ms
Speed: 5.7ms preprocess, 6.2ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
